# Repository Tutorial: Getting Started

This notebook walks through the key components of our repository, showing how to work with models, datasets, and evaluation utilities.

## Setup

First, let's import necessary modules and set up our environment.

In [33]:
# Import necessary modules and enable caching
from src import cache
cache.enable()
from src.model.model import Model
from key_handler import KeyHandler
from eval_datasets.types.gsm8k import GSM8KDataset
from copy import deepcopy
import os
import pprint

Redis server connected.


## 1. Getting Started with Basic Model and Dataset Interactions

This section demonstrates how to load a model and dataset, and run basic inferences.

In [34]:
# Set environment variables (including OPENAI_API_KEY)
KeyHandler.set_env_key()

# Load a model
model = Model.load_model('openai/gpt-4o-mini-2024-07-18')

# Load a dataset (GSM8K for math reasoning tasks)
dataset = GSM8KDataset()

# Get an example from the dataset
example = deepcopy(dataset[0])

pprint.pprint(example)

{'answer': '10',
 'answerKey': '10',
 'dataset_type': 'gsm8k',
 'fs_cot_messages': [{'content': 'You are a helpful AI assistant that will '
                                 'answer reasoning questions. You will reason '
                                 'step by step and you will always say at the '
                                 'end $\\boxed{your answer}$". You must end '
                                 'your response with "\\boxed{your answer}" '
                                 'everytime!',
                      'role': 'system'},
                     {'content': 'Solve the following math problem.  Explain '
                                 'your reasoning step by step.  When you are '
                                 'finished, box your final answer: '
                                 '$\\boxed{{your answer}}$.\n'
                                 '            \n'
                                 'Problem:\n'
                                 'There are 15 trees in the grove. Gro

### 1.1 Understanding Prompting Strategies

Let's examine two different prompting approaches:
- Zero-shot Chain-of-Thought (CoT): Encourages the model to think step-by-step
- Direct Answer: Asks the model to provide just the answer

In [35]:
# Get the prompts for both approaches
zs_cot_prompt = example['zs_cot_messages']
zs_directanswer_prompt = example['zs_cotless_messages']

In [36]:
# Print the CoT system prompt
print("Chain-of-Thought System Prompt:")
print(zs_cot_prompt[0]['content'])

Chain-of-Thought System Prompt:
You are a helpful AI assistant that will answer reasoning questions. You will reason step by step and you will always say at the end $\boxed{your answer}$". You must end your response with "\boxed{your answer}" everytime!


In [37]:
# Print the Direct Answer system prompt
print("Direct Answer System Prompt:")
print(zs_directanswer_prompt[0]['content'])

Direct Answer System Prompt:
You are a helpful AI assistant that will answer reasoning questions. You will only say "\boxed{your answer}". You must end your response with $\boxed{your answer}$ everytime!


### 1.2 Running Inference

In [38]:
# Run inference with Chain-of-Thought prompting
models_cot_response = model.parse_out(model.inference(zs_cot_prompt))
print(f'Model\'s CoT Response:\n{models_cot_response[0]}\n')


Model's CoT Response:
Let's solve the problem step by step.

1. **Initial Number of Pencils:** Anthony starts with 50 pencils.

2. **Pencils Given to Brandon:** Anthony gives half of his pencils to Brandon. 
   - Half of 50 is calculated as \( \frac{1}{2} \times 50 = 25 \).
   - So, Anthony gives 25 pencils to Brandon.

3. **Pencils Remaining After Giving to Brandon:** 
   - After giving 25 pencils to Brandon, the number of pencils Anthony has left is:
   \( 50 - 25 = 25 \).

4. **Pencils Given to Charlie:** Now, Anthony gives 3/5 of his remaining pencils (which is 25) to Charlie.
   - To find out how many pencils that is, we calculate \( \frac{3}{5} \times 25 \):
     - First, we calculate \( \frac{25}{5} = 5 \).
     - Then, multiply by 3: \( 3 \times 5 = 15 \).
   - Thus, Anthony gives 15 pencils to Charlie.

5. **Final Number of Pencils Kept by Anthony:**
   - Now, we find out how many pencils Anthony has left after giving 15 pencils to Charlie:
   \( 25 - 15 = 10 \).

Therefore, A

In [39]:
# Run inference with Direct Answer prompting
models_directanswer_response = model.parse_out(model.inference(zs_directanswer_prompt))

# Notice that this isn't a direct answer (the model gives a chain of thought without being asked). We will fix in the following sections with partial completions.
print(f'Model\'s Direct Answer Response:\n{models_directanswer_response[0]}')

Model's Direct Answer Response:
Anthony started with 50 pencils. 

First, he gave half of his pencils to Brandon:

\[
\text{Pencils given to Brandon} = \frac{1}{2} \times 50 = 25
\]

After giving pencils to Brandon, he has:

\[
\text{Remaining pencils} = 50 - 25 = 25
\]

Next, he gave 3/5 of the remaining pencils to Charlie:

\[
\text{Pencils given to Charlie} = \frac{3}{5} \times 25 = 15
\]

After giving pencils to Charlie, he has:

\[
\text{Remaining pencils} = 25 - 15 = 10
\]

Thus, the number of pencils Anthony kept is:

\[
\boxed{10}
\]


### 1.3 Evaluating Responses

In [40]:
# Evaluate both responses
examples_cot_metrics = dataset.evaluate_response(models_cot_response, example)
examples_directanswer_metrics = dataset.evaluate_response(models_directanswer_response, example)

# Print evaluation results
print(f"The correct answer: {example['answer']}")

The correct answer: 10


In [41]:
# Extract the answer span from the CoT response
answer_span_in_cot_response = examples_cot_metrics[0]["answer_span"]
print(f'CoT\'s extracted answer: {examples_cot_metrics[0]["model_response"][answer_span_in_cot_response[0]:answer_span_in_cot_response[1]]}')

CoT's extracted answer: 10


In [42]:
# Check if the answers are correct
print(f"CoT was correct: {examples_cot_metrics[0]['correct']}")
print(f"Direct Answer was correct: {examples_directanswer_metrics[0]['correct']}")

CoT was correct: True
Direct Answer was correct: True


## 2. Working with Partial Completions

Some models support partial completions, which allow you to "warm start" the generation by providing part of the response.

In [43]:
# Load a model that supports partial completions
model = Model.load_model('anthropic/claude-3-haiku-20240307')
dataset = GSM8KDataset()
example = deepcopy(dataset[0])

# Get the Zero-shot CoT prompt
zs_cot_prompt = deepcopy(example['zs_cot_messages'])

# Create a partial completion by adding an assistant message with beginning text
zs_cot_prompt.append({
    'role': 'assistant',
    'content': 'Oh wow. This is a difficult problem. Let me start by'
})

# The model will continue from this partial completion
models_completion_of_partial_response = model.parse_out(model.inference(zs_cot_prompt))

print('Partial Prompt: Oh wow. This is a difficult problem. Let me start by')
print(f"Model's output: {models_completion_of_partial_response[0]}")

Partial Prompt: Oh wow. This is a difficult problem. Let me start by
Model's output:  breaking it down step by step:
1) Anthony had 50 pencils initially.
2) He gave 1/2 of his pencils to Brandon. So, he gave away 1/2 * 50 = 25 pencils to Brandon.
3) Now, Anthony has 50 - 25 = 25 pencils remaining.
4) Anthony then gave 3/5 of the remaining 25 pencils to Charlie. So, he gave away 3/5 * 25 = 15 pencils to Charlie.
5) Finally, Anthony kept the remaining pencils. The remaining pencils are 25 - 15 = 10 pencils.

$\boxed{10 pencils}$


### 2.1 Guiding Direct Answers with Partial Completions

We can use partial completions to help models avoid generating chains of thought, especially for math questions.

In [44]:
# Get the direct answer prompt
zs_directanswer_prompt = example['zs_cotless_messages']

# Run inference without partial completion
orig_directanswer_response = model.parse_out(model.inference(zs_directanswer_prompt))

# Print original prompt and response
print("System prompt:")
print(zs_directanswer_prompt[0]["content"])
print("\nPrompt:")
print(zs_directanswer_prompt[1]['content'])
print("\nModel's original output:")
print(orig_directanswer_response[0])

# Add partial completion to guide the model to give a boxed answer
zs_directanswer_prompt.append({
    'role': 'assistant',
    'content': '$\\boxed{'
})

# Run inference with the partial completion
new_directanswer_response = model.parse_out(model.inference(zs_directanswer_prompt))
print("\nNew model output with the partial completion $\\boxed{")
print(new_directanswer_response[0])

System prompt:
You are a helpful AI assistant that will answer reasoning questions. You will only say "\boxed{your answer}". You must end your response with $\boxed{your answer}$ everytime!

Prompt:
Solve the following math problem. Box your final answer: $\boxed{your answer}$.

Problem:
Anthony had 50 pencils. He gave 1/2 of his pencils to Brandon, and he gave 3/5 of the remaining pencils to Charlie. He kept the remaining pencils. How many pencils did Anthony keep?

Remember to box your final answer via $\boxed{your answer}$.
            

Model's original output:
Okay, let's solve this step-by-step:
1) Anthony had 50 pencils initially.
2) He gave 1/2 of his pencils to Brandon. 1/2 of 50 is 25, so he gave Brandon 25 pencils.
3) He had 50 - 25 = 25 pencils remaining.
4) He then gave 3/5 of the remaining 25 pencils to Charlie. 3/5 of 25 is 15, so he gave Charlie 15 pencils.
5) He had 25 - 15 = 10 pencils remaining, which he kept.
Therefore, the final answer is:
$\boxed{10}$

New model o

## 3. Rollouts and Batching

This section demonstrates how to evaluate a model using multiple samples and handle batched requests.

In [49]:
from pathlib import Path
from experiments.utils import rollout
from eval_datasets.types.musr import MuSRDataset

# Load model and dataset
model = Model.load_model('openai/gpt-4o-mini-2024-07-18')

# The MuSR dataset has a custom loader as it's a third-party file
dataset = MuSRDataset(str(Path().resolve().parent / 'eval_datasets/thirdparty/musr/murder_mystery.json'))
example = deepcopy(dataset[10])

# Get the prompts
zs_cot_prompt = example['zs_cot_messages']
zs_directanswer_prompt = example['zs_cotless_messages']

# Set up rollout parameters
NUMBER_OF_SAMPLES = 10  # Number of samples to generate
BATCH_SIZE = 4          # Number of requests to process in parallel

# Set generation parameters
max_completion_length = 4096
temperature = 0.7
top_p = 0.95

### 3.1 Running Rollouts

In [51]:
# Run rollouts for Chain-of-Thought prompting
cot_accuracy, cot_unparsable_rate, cot_metrics = rollout(
    model,
    zs_cot_prompt,
    example,
    num_rollouts=NUMBER_OF_SAMPLES,
    batch_size=BATCH_SIZE,
    completion_length=max_completion_length,
    temperature=temperature,
    top_p=top_p
)

# Run rollouts for Direct Answer prompting
directanswer_accuracy, directanswer_unparsable_rate, directanswer_metrics = rollout(
    model,
    zs_directanswer_prompt,
    example,
    num_rollouts=NUMBER_OF_SAMPLES,
    batch_size=BATCH_SIZE,
    completion_length=max_completion_length,
    temperature=temperature,
    top_p=top_p
)

# Compare results
print(f'Direct Answer vs CoT Accuracy (Unparseable Rate)')
print(f'DA: {directanswer_accuracy:.2f} ({directanswer_unparsable_rate:.2f})')
print(f'CoT: {cot_accuracy:.2f} ({cot_unparsable_rate:.2f})')

Direct Answer vs CoT Accuracy (Unparseable Rate)
DA: 0.00 (0.00)
CoT: 0.58 (0.00)


## 4. Using Hosted vLLM

You can use the repository with models hosted on vLLM endpoints.

In [52]:
# Example of loading a model from a vLLM endpoint
vllm_model = Model.load_model("vllm_endpoint/http://127.0.0.1:60271/v1/completions<model>deepseek-ai/DeepSeek-R1-Distill-Llama-70B")

# The rest of the API usage remains the same
# You can use this model just like any other model loaded above

## 5. Adding a New Dataset

This section demonstrates how to add a new dataset to the repository. We'll use SocialIQA as an example.

In [55]:
import random
from datasets import load_dataset
from eval_datasets import ReasoningDataset, CSQADataset

class SocialIQADataset(ReasoningDataset):
    average_token_len = 300
    
    def __init__(self, path_or_url='social_i_qa', split='validation', *args, **kwargs):
        super().__init__(path_or_url + ':' + split, *args, **kwargs)
    
    @classmethod
    @property
    def dataset_type(cls):
        return cls.dataset_types.socialiqa
    
    def load_dataset(self, path_or_url):
        examples = []
        dataset_url, split = path_or_url.split(':')
        dataset = [x for x in load_dataset(dataset_url, trust_remote_code=True)[split]]
        
        for ex in dataset:
            # Format choices for the model
            choices = {'text': [ex['answerA'], ex['answerB'], ex['answerC']], 'label': ['A', 'B', 'C']}
            answer_index = int(ex['label']) - 1
            answer = [ex['answerA'], ex['answerB'], ex['answerC']][answer_index]
            
            # Create prompts for both CoT and direct answer approaches
            zs_cot_prompt = self.basic_prompt(ex["context"], self.format_choices(choices))
            zs_cotless_prompt = self.basic_prompt(ex["context"], self.format_choices(choices), direct=True)
            
            examples.append({
                **ex,
                'dataset_type': self.dataset_types.socialiqa,
                'prompt_parts': {
                    'zs_cot_prompt': zs_cot_prompt,
                    'zs_cotless_prompt': zs_cotless_prompt,
                    'cot_system_prompt': self.default_sys_mc_cot_prompt,
                    'cotless_system_prompt': self.default_sys_mc_cotless_prompt
                },
                'choices': choices,
                'answer': answer,
                'answer_index': answer_index,
                'answer_choice_tokens': ['A', 'B', 'C'],
                'answerKey': ['A', 'B', 'C'][answer_index],
                'og_question': ex['question'],
                'question': ex['context'] + '\n\n' + ex['question'],
            })
        return examples
    
    @classmethod
    def evaluate_response(
            cls,
            model_responses,
            example,
            randomly_select_when_unparsable: bool = False,
            *args, **kwargs
    ):
        # Re-use the evaluation function from CSQA dataset
        return CSQADataset.evaluate_response(model_responses, example, randomly_select_when_unparsable, *args, **kwargs)
    
    @classmethod
    def custom_evaluate_response(cls, model_responses, example, *args, **kwargs):
        # Custom evaluation logic specific to this dataset (if needed)
        return None

### 5.1 Testing the New Dataset

In [56]:
# Test the new dataset
from torch.utils.data import DataLoader

# Create an instance of the dataset
dataset = SocialIQADataset()
ex = dataset[0]

# Test the evaluation function with sample responses
responses = [
    'I think 1 2 the answer is 1',
    'ANSWER: 2\n\nBecause of..\n\nSo answer 1',
    'I think because...\n\nANSWER: 3',
    'I think because...\n\nANSWER: 1 or 2'
]

# Evaluate the responses
metrics = dataset.evaluate_response(responses, ex)

# Print the results
print([x['model_answer'] for x in metrics])
print(ex['messages'][0]['content'])

[None, None, None, None]
You are a helpful AI assistant that will answer reasoning questions. You may reason over the question but you will always say at the end "Answer: <Your Answer Letter Choice>". You must only pick one answer and you must end your response with "Answer: <Your Answer Letter Choice>" everytime!


## Key Components in Dataset Implementation

When adding a new dataset, make sure to:

1. **Load the dataset**: Pull data from Huggingface or other sources
2. **Format prompts**: Set up both zero-shot CoT and direct answer prompts
3. **Define answer parsers**: 
   - For multiple-choice datasets, you can reuse parsers from similar datasets
   - Implement custom_evaluate_response for dataset-specific logic
4. **Prepare example structures**: Include all necessary fields for evaluation

This approach provides a flexible framework for evaluating different models on various benchmarks.